Create our keys and retrieve the oracle account already set up

In [2]:
from multisigcore import *
from multisigcore.providers.electrum import ElectrumService
from multisigcore.oracle import PersonalInformation, OracleUnknownKeychainException
from multisigcore.hierarchy import MasterKey, MultisigAccount
from pycoin.services import get_tx_db
from pycoin.services import insight
from pycoin.serialize import b2h, h2b

secrets = [u'aaa4', u'bbb4']
keys = [MasterKey.from_master_secret(x).bip44_account(0) for x in secrets]
account = MultisigAccount(keys, complete=False)
oracle = Oracle(account, tx_db=get_tx_db())
oracle.verbose = 0
parameters = {"levels": [ {
                          "asset": "BTC",
                          "period": 3600,
                          "value": 1.0
                      }]}
try: oracle.get()
except OracleUnknownKeychainException: oracle.create(parameters, PersonalInformation())

print("http://btc.blockr.io/address/info/" + account.address(0))
account.set_lookahead(0)
service = insight.InsightService('https://insight.bitpay.com/')
#service = ElectrumService("electrum.no-ip.org", 50002)
account._provider = service
print(account.balance())
print(oracle._url())

http://btc.blockr.io/address/info/3EmiFHM7xVVDiK7TiKUDJCWgbJFV37ZLbo
49000
https://s.digitaloracle.co/keychains/feca3955-ab21-57e4-ae6b-4e5e8a29a549


Create a transaction spending funds previously sent, then sign

In [5]:
tx = account.tx([('17HXqN4Prf4o6sjAm2X43gLhwwzCrhAauZ', 50000)])
account.sign(tx)
res = oracle.sign(tx, "spend003")
print(res)
print(res.transaction.as_hex())

{'spend_id': u'1212b4b0-dcb3-11e4-bbd3-09cbbe730f06', 'now': u'2015-04-06T23:17:19Z', 'deferral': None, 'transaction': Tx [e2645d47596c315115968903add35b959bfaead735236be4b7febcbd2d09f936] (v:1) [TxIn<d779e67d1892a29f5c71ae223d61658192174ecabfc1d3c6eb8829d5753b6c89[0] "OP_0 30450221008c6af1717e313ec99a3a60018a7858455aeb7dd254f6a8b8ec91b59550848a95022035a56b348df2db9c18403bd9b823ee074dacc13282ced79488384e6e3f14d39901 30450221009e1417382f63b1902a5c8be88d11273679e33e66a05cb37d12d8e344faddc0b20220601c91586920623acc33eaa6643eeb7acc35d0d08658a33bf12620545701dac801 52210284840b6315fed9a522d142a56d74c43f371a78aa0d41d6a77d83da9f12144740210347b250c317b01a6ea743324589cf9d14cb499c0d6837aa8acffadff66070e6ff210350be0dea89f304c791fee469454fcccb3f9a5bb5a91310dda726f533881b354253ae">] [TxOut<0.5 mbtc "OP_DUP OP_HASH160 44f1261da5afc9fe7dc8980dd8ab23e391504fc3 OP_EQUALVERIFY OP_CHECKSIG">, TxOut<0.49 mbtc "OP_HASH160 6c42dd8d12039387318e845ac3ac27419224fe0c OP_EQUAL">]}
0100000001896c3b75d52988ebc6d3c1b